# Importando las librerías.

In [1]:
import findspark
from pyspark import SparkContext, SparkConf, SQLContext
import csv
import geopandas as gpd
from pyspark.sql.functions import substring, expr
from shapely import wkt
import pandas as pd
findspark.init()  # Con este no me tira error de JVM.

# Abriendo los archivos.

In [2]:
# Se setea el Master y se le da nombre a la aplicacion.
conf = SparkConf().setMaster('local').setAppName('Tarea Analisis de BigData')

In [3]:
# Se inicia el cluster Spark.
sc = SparkContext.getOrCreate(conf=conf)

In [4]:
# Se inicia SQLContext desde el cluster de Spark.
sqlContext = SQLContext(sc)

In [5]:
# Carpeta donde estan guardados los archivos
FilePath = '/home/rafa/Dropbox/Linux_MDS/BDAnalytics/sprint1/data'

In [6]:
# Nombres de los archivo.
FileName1 = 'wifi_2017.csv'
FileName2 = 'wifi_2018.csv'
FileName3 = 'wifi_2019.csv'

In [7]:
# Combinando
FullPath1 = FilePath + '/' + FileName1
FullPath2 = FilePath + '/' + FileName2
FullPath3 = FilePath + '/' + FileName3


In [8]:
# Dataframes
df_2017 = sqlContext.read.csv(FullPath1, header=True)
df_2018 = sqlContext.read.csv(FullPath2, header=True)
df_2019 = sqlContext.read.csv(FullPath3, header=True)

# Mostrando los archivos.

In [9]:
# Mostrando los data frame
df_2017.show()

+---+-----------+------------+-----------+-----------+------+----------+----------+
| id|data_source|       bssid|        lat|        lon| range|   updated|   created|
+---+-----------+------------+-----------+-----------+------+----------+----------+
|  1|          0|0001E3C290F9|45.22128388|16.54838246|139.01|1428513555|1428513555|
|  2|          0|000C421F65E9|45.22027969|16.54707052|140.00|1428513555|1428513555|
|  3|          0|000C422377FC|45.20828617|16.90385474|140.00|1428513555|1428513555|
|  4|          0|000C422377F4|45.20799464|16.90336159|140.00|1428513555|1428513555|
|  5|          0|000C422377C5|45.17570261|16.81720250|140.00|1428513555|1428513555|
|  6|          0|000C42630AED|45.18732229|16.83431923|140.00|1428513555|1428513555|
|  7|          0|000C42630AEC|45.18054666|16.81053529|140.00|1428513555|1428513555|
|  8|          0|0005595C556A|45.18191138|16.80946429|140.00|1428513555|1428513555|
|  9|          0|000C421BB4FE|45.20791481|16.90272641|140.00|1428513555|1428

In [10]:
df_2018.show()

+-------+------------+-----------+-----------+----------+----+
|     id|       bssid|        lat|        lon|   updated|data|
+-------+------------+-----------+-----------+----------+----+
|1773306|F83DFFBBBA29|59.31152020|18.07471230|1428513555|   0|
|1773307|0026180A9583|59.29648818|18.06929944|1428513555|   0|
|1773308|00248C3B965B|59.30886047|18.07669099|1428513555|   0|
|1773309|00248CAAF77A|59.29378075|18.05087886|1428513555|   0|
|1773310|4C60DE7CC982|59.31155087|18.07525706|1428513555|   0|
|1773311|00248C93A4FC|59.30421726|18.07856642|1428513555|   0|
|1773312|A021B74D1A61|59.29762960|18.07197083|1428513555|   0|
|1773313|0001387568D4|59.29430646|18.04973913|1428513555|   0|
|1773315|0025BC8DA6FC|59.31209248|18.07632087|1428513555|   0|
|1773317|2837374A8235|59.30896188|18.07752865|1428513555|   0|
|1773318|0016A61817FA|59.29355670|18.05207263|1428513555|   0|
|1773319|00223F9C5739|59.29361025|18.05284041|1428513555|   0|
|1773320|0025BC8DA6FB|59.31190825|18.07605543|142851355

In [11]:
df_2019.show()

+---+------------+-----------+-----------+----------+----+
| id|       bssid|        lat|        lon|   updated|data|
+---+------------+-----------+-----------+----------+----+
|  1|0001E3C290F9|45.22128388|16.54838246|1428513555|   0|
|  2|000C421F65E9|45.22027969|16.54707052|1428513555|   0|
|  3|000C422377FC|45.20828617|16.90385474|1428513555|   0|
|  4|000C422377F4|45.20799464|16.90336159|1428513555|   0|
|  5|000C422377C5|45.17570261|16.81720250|1428513555|   0|
|  6|000C42630AED|45.18732229|16.83431923|1428513555|   0|
|  7|000C42630AEC|45.18054666|16.81053529|1428513555|   0|
|  8|0005595C556A|45.18191138|16.80946429|1428513555|   0|
|  9|000C421BB4FE|45.20791481|16.90272641|1428513555|   0|
| 10|000C422CC930|45.19232675|16.84438858|1428513555|   0|
| 11|0005595C7B59|45.18621844|16.83064594|1428513555|   0|
| 12|000C420CCF25|45.17452761|16.81220048|1428513555|   0|
| 13|000559604EC0|45.20764763|16.89996500|1428513555|   0|
| 14|000C42654000|45.17547062|16.81085728|1428513555|   

In [12]:
# Muestro el esquema de los dataframes.
df_2017.printSchema()

root
 |-- id: string (nullable = true)
 |-- data_source: string (nullable = true)
 |-- bssid: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lon: string (nullable = true)
 |-- range: string (nullable = true)
 |-- updated: string (nullable = true)
 |-- created: string (nullable = true)



In [13]:
df_2018.printSchema()

root
 |-- id: string (nullable = true)
 |-- bssid: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lon: string (nullable = true)
 |-- updated: string (nullable = true)
 |-- data: string (nullable = true)



In [14]:
df_2019.printSchema()

root
 |-- id: string (nullable = true)
 |-- bssid: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lon: string (nullable = true)
 |-- updated: string (nullable = true)
 |-- data: string (nullable = true)



# Trabajo de orden y unión de datasets

In [15]:
# Cambiando el nombre a la columna data_source por data
df_2017 = df_2017.withColumnRenamed("data_source","data")

df_2017.printSchema()

df_2017.show()

root
 |-- id: string (nullable = true)
 |-- data: string (nullable = true)
 |-- bssid: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lon: string (nullable = true)
 |-- range: string (nullable = true)
 |-- updated: string (nullable = true)
 |-- created: string (nullable = true)

+---+----+------------+-----------+-----------+------+----------+----------+
| id|data|       bssid|        lat|        lon| range|   updated|   created|
+---+----+------------+-----------+-----------+------+----------+----------+
|  1|   0|0001E3C290F9|45.22128388|16.54838246|139.01|1428513555|1428513555|
|  2|   0|000C421F65E9|45.22027969|16.54707052|140.00|1428513555|1428513555|
|  3|   0|000C422377FC|45.20828617|16.90385474|140.00|1428513555|1428513555|
|  4|   0|000C422377F4|45.20799464|16.90336159|140.00|1428513555|1428513555|
|  5|   0|000C422377C5|45.17570261|16.81720250|140.00|1428513555|1428513555|
|  6|   0|000C42630AED|45.18732229|16.83431923|140.00|1428513555|1428513555|
|  7|   0

In [16]:
# Noto que hay columnas extras en un dataset, en primera instancia las elimino.
df_2017 = df_2017.drop('range', 'created')

df_2017.printSchema()

df_2017.show()

root
 |-- id: string (nullable = true)
 |-- data: string (nullable = true)
 |-- bssid: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lon: string (nullable = true)
 |-- updated: string (nullable = true)

+---+----+------------+-----------+-----------+----------+
| id|data|       bssid|        lat|        lon|   updated|
+---+----+------------+-----------+-----------+----------+
|  1|   0|0001E3C290F9|45.22128388|16.54838246|1428513555|
|  2|   0|000C421F65E9|45.22027969|16.54707052|1428513555|
|  3|   0|000C422377FC|45.20828617|16.90385474|1428513555|
|  4|   0|000C422377F4|45.20799464|16.90336159|1428513555|
|  5|   0|000C422377C5|45.17570261|16.81720250|1428513555|
|  6|   0|000C42630AED|45.18732229|16.83431923|1428513555|
|  7|   0|000C42630AEC|45.18054666|16.81053529|1428513555|
|  8|   0|0005595C556A|45.18191138|16.80946429|1428513555|
|  9|   0|000C421BB4FE|45.20791481|16.90272641|1428513555|
| 10|   0|000C422CC930|45.19232675|16.84438858|1428513555|
| 11|   0|0

In [17]:
# Ahora ordeno las columnas para que todas tengan el mismo orden.
df_2017 = df_2017.select('id','bssid','lat','lon', 'updated', 'data')

df_2017.printSchema()

df_2017.show()

root
 |-- id: string (nullable = true)
 |-- bssid: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lon: string (nullable = true)
 |-- updated: string (nullable = true)
 |-- data: string (nullable = true)

+---+------------+-----------+-----------+----------+----+
| id|       bssid|        lat|        lon|   updated|data|
+---+------------+-----------+-----------+----------+----+
|  1|0001E3C290F9|45.22128388|16.54838246|1428513555|   0|
|  2|000C421F65E9|45.22027969|16.54707052|1428513555|   0|
|  3|000C422377FC|45.20828617|16.90385474|1428513555|   0|
|  4|000C422377F4|45.20799464|16.90336159|1428513555|   0|
|  5|000C422377C5|45.17570261|16.81720250|1428513555|   0|
|  6|000C42630AED|45.18732229|16.83431923|1428513555|   0|
|  7|000C42630AEC|45.18054666|16.81053529|1428513555|   0|
|  8|0005595C556A|45.18191138|16.80946429|1428513555|   0|
|  9|000C421BB4FE|45.20791481|16.90272641|1428513555|   0|
| 10|000C422CC930|45.19232675|16.84438858|1428513555|   0|
| 11|000559

In [18]:
# Ahora uno los df_2017, df_2018 y df_2019
df_2017_2018_2019 = ((df_2017.union(df_2018)).union(df_2019)).distinct()
df_2017_2018_2019.show(truncate=False)

In [ ]:
# Elimino las columnas innecesarias.
df_2017_2018_2019 = df_2017_2018_2019.drop('updated', 'data')

In [ ]:
df_all = df_2017_2018_2019

# Ahora creo el df solo con la RM según coordenadas de google maps.
df_stgo = df_all.filter((df_all.lat >= -33.65) &
                        (df_all.lat <= -33.28) &
                        (df_all.lon >= -70.81) &
                        (df_all.lon <= -70.50))
df_stgo.show(truncate=False)

+-------+------------+------------+------------+
|id     |bssid       |lat         |lon         |
+-------+------------+------------+------------+
|8849791|1CE6C7F010E6|-33.39275272|-70.57426126|
|8849792|1CE6C7F010E5|-33.39250351|-70.57437245|
|8850002|001EE53444A9|-33.39918647|-70.58116012|
|8850139|64E95033D911|-33.39342478|-70.57411691|
|8850625|FC94E30C3C64|-33.40811106|-70.58709986|
|8851402|34DBFDBFF298|-33.39346036|-70.57409400|
|8991482|001DCF203FD1|-33.40576570|-70.56381641|
|8991542|2CAB2585EF3F|-33.40188118|-70.56105160|
|8991749|004F621D5873|-33.39870261|-70.55496093|
|8992564|40CBA8CCBD9D|-33.41227703|-70.57368460|
|8992611|B4750E38BC6E|-33.40116664|-70.56038709|
|8993378|14ABF0DD5A70|-33.42087583|-70.56632524|
|8994248|70105C7CD8F0|-33.41674528|-70.57156504|
|8994782|FC94E33EB2E5|-33.40310366|-70.56229882|
|8994818|001DCE35D79D|-33.41167474|-70.57389409|
|8994939|14ABF0DA0A00|-33.40700644|-70.56115041|
|8995096|14ABF0DA3B20|-33.41186394|-70.57383565|
|8995103|FC8B97CB0F9

In [ ]:
# Credo el df final de Stgo.
# Separo la columna bssid en una que contendrá Media_mac y otra que contendrá
# Id_fabricante.
df_stgo = df_stgo.withColumn("Id_fabricante", expr("substring(bssid,1,length(bssid)-6)")).withColumn(
    "Media_mac", expr("substring(bssid,7,length(bssid)-6)")).drop("bssid")
df_stgo.show()

+-------+------------+------------+-------------+---------+
|     id|         lat|         lon|Id_fabricante|Media_mac|
+-------+------------+------------+-------------+---------+
|8849791|-33.39275272|-70.57426126|       1CE6C7|   F010E6|
|8849792|-33.39250351|-70.57437245|       1CE6C7|   F010E5|
|8850002|-33.39918647|-70.58116012|       001EE5|   3444A9|
|8850139|-33.39342478|-70.57411691|       64E950|   33D911|
|8850625|-33.40811106|-70.58709986|       FC94E3|   0C3C64|
|8851402|-33.39346036|-70.57409400|       34DBFD|   BFF298|
|8991482|-33.40576570|-70.56381641|       001DCF|   203FD1|
|8991542|-33.40188118|-70.56105160|       2CAB25|   85EF3F|
|8991749|-33.39870261|-70.55496093|       004F62|   1D5873|
|8992564|-33.41227703|-70.57368460|       40CBA8|   CCBD9D|
|8992611|-33.40116664|-70.56038709|       B4750E|   38BC6E|
|8993378|-33.42087583|-70.56632524|       14ABF0|   DD5A70|
|8994248|-33.41674528|-70.57156504|       70105C|   7CD8F0|
|8994782|-33.40310366|-70.56229882|     

# Archivo de texto

In [ ]:
# Ahora trabajo con el archivo de texto creando un diccionario
result = dict()
for lig in open('/home/rafa/Dropbox/Linux_MDS/BDAnalytics/sprint1/data/oui.txt'):
    if 'base 16' in lig:
        num, sep, txt = lig.strip().partition('(base 16)')
        result[num.strip()] = txt.strip()

In [ ]:
# Transformo el diccionario en csv para mejor manipulacion
with open('/home/rafa/Dropbox/Linux_MDS/BDAnalytics/sprint1/oui.csv',
          'w') as f:
    w = csv.writer(f)
    w.writerows(result.items())


In [ ]:
# Creo el df
df_oui = sqlContext.read.csv('/home/rafa/Dropbox/Linux_MDS/BDAnalytics/'
                             'sprint1/oui.csv',
                             header=False)

In [ ]:
# Despliego el dataframe, es importante saber que los ultimos 6 dígitos del
# bssid corresponden al Id indicado como _c0
df_oui.show(truncate=False)

+------+------------------------------------------------------------+
|_c0   |_c1                                                         |
+------+------------------------------------------------------------+
|002272|American Micro-Fuel Device Corp.                            |
|00D0EF|IGT                                                         |
|086195|Rockwell Automation                                         |
|F4BD9E|Cisco Systems, Inc                                          |
|5885E9|Realme Chongqing MobileTelecommunications Corp Ltd          |
|BC2392|BYD Precision Manufacture Company Ltd.                      |
|405582|Nokia                                                       |
|A4E31B|Nokia                                                       |
|D89790|Commonwealth Scientific and Industrial Research Organisation|
|883A30|Aruba, a Hewlett Packard Enterprise Company                 |
|B8A58D|Axe Group Holdings Limited                                  |
|50CEE3|Gigafirm.co.

# Shapefile

In [ ]:
# Este archivo contiene todo lo solictado, salvo la ciudad.
Manzana_Precensal = gpd.read_file('/home/rafa/Dropbox/Linux_MDS/BDAnalytics/'
                                  'sprint1/data/'
                                  'Manzana_Precensal.shp')

In [ ]:
# Elimino columnas innecesarias.
# COD_ZON corresponde a Zona Censal.
# COD_ENT corrresponde a Manzana Censal.
# DES_COMU a la comuna.
Manzana_Precensal = Manzana_Precensal.drop(['DES_REGI', 'MANZENT', 'COMUNA',
                                            'PROVINCIA', 'DES_PROV', 'REGION',
                                            'COD_DIS'],
                                           axis=1)
Manzana_Precensal

,COD_ZON,COD_ENT,DES_COMU,geometry
0,3,1,CERRILLOS,"POLYGON ((-70.72166 -33.47385, -70.72197 -33.4..."
1,3,4,CERRILLOS,"POLYGON ((-70.72136 -33.47420, -70.72160 -33.4..."
2,3,5,CERRILLOS,"POLYGON ((-70.72105 -33.47455, -70.72124 -33.4..."
3,3,8,CERRILLOS,"POLYGON ((-70.72079 -33.47509, -70.72089 -33.4..."
4,3,9,CERRILLOS,"POLYGON ((-70.72052 -33.47563, -70.72052 -33.4..."
...,...,...,...,...
53048,1,504,PEÑAFLOR,"POLYGON ((-70.87621 -33.61954, -70.87751 -33.6..."
53049,1,503,PEÑAFLOR,"POLYGON ((-70.87723 -33.61920, -70.87741 -33.6..."
53050,1,502,PEÑAFLOR,"POLYGON ((-70.87713 -33.61883, -70.87731 -33.6..."
53051,1,505,PEÑAFLOR,"POLYGON ((-70.87556 -33.61840, -70.87561 -33.6..."


# Construyendo los futures

## Fabricante

In [ ]:
# Uno los df_stgo y df_oui a través de un join y además le solicito que lo haga
# en donde Id_fabricante sea idéntico a _c0 del df_oui.
df_stgo = df_stgo.join(df_oui).where(df_stgo["Id_fabricante"] == df_oui["_c0"])
df_stgo = df_stgo.drop('_c0')
df_stgo = df_stgo.withColumnRenamed('_c1', 'Fabricante')
print('El dataframe con el primer future es el siguiente:\n')
df_stgo.show(truncate=False)

In [ ]:
# Información geográfica (ciudad, comuna, zona censal, manzana)
# Transformo df_stgo a pandas para poder trabajarlo con geopandas
df_stgo_pandas = df_stgo.toPandas()

# Ahora creo un geopandas para indicarle los points de lat y lon.
df_stgo_geop = gpd.GeoDataFrame(df_stgo_pandas,
                                geometry=gpd.points_from_xy
                                (df_stgo_pandas.lon, df_stgo_pandas.lat))

In [ ]:
df_stgo_geop = df_stgo_geop.drop(columns=['lat', 'lon'])

# Le indico el CRS para que quede igual a Manzana_Precensal.
df_stgo_geop.crs = 'EPSG:4674'

In [ ]:
# Ahora hago un join con geopandas entre df_stgo_geop y Manzana_Precensal.
join_stgo_manzana = gpd.sjoin(df_stgo_geop, Manzana_Precensal, op='within', 
                              how='inner')

In [ ]:
# Ahora hago un join con geopandas entre df_stgo_geop y Manzana_Precensal.
join_stgo_manzana = gpd.sjoin(df_stgo_geop, Manzana_Precensal, op='within', 
                              how='inner')

join_stgo_manzana = join_stgo_manzana.drop(columns=['index_right'])
join_stgo_manzana['str_geom'] = join_stgo_manzana.geometry.apply(lambda x: wkt.dumps(x))
join_stgo_manzana = (join_stgo_manzana.drop(columns=['geometry'])
                     ).rename(columns={'str_geom': 'geometry',
                                       'COD_ZON': 'Zona_Censal',
                                       'COD_ENT': 'Manzana_Censal',
                                       'DES_COMU': 'Comuna'})
# join_stgo_manzana.rename(columns={'str_geom': 'geometry'})

# Ahora lo vuelvo a pasar a pyspark.
fab_info_geo = sqlContext.createDataFrame(join_stgo_manzana)

fab_info_geo.show(truncate=False)